# 1. 브론즈(Bronze) 테이블의 개념

Oracle AI Data Platform(AIDP)는 메달리온(Medallion) 아키텍처, 브론즈 → 실버 → 골드 구조를 기반으로 설계되어 있습니다.  
그중 **브론즈(Bronze) 레이어**는 가장 처음 데이터를 받아서, 원본에 가깝게 쌓아두는 단계입니다.

브론즈 레이어의 핵심 개념은 다음과 같습니다.

- **역할**  
  - 외부에서 들어온 데이터를 *거의 그대로* 저장  
  - 나중에 문제가 생겼을 때 원본을 다시 확인할 수 있는 원천 데이터 보관소
- **데이터 출처 예시**
  - CSV 파일 예: CRM 시스템에서 떨어지는 고객 데이터
  - 애플리케이션 로그
  - 외부 시스템에서 가져온 정기 dump 파일 등
- **브론즈의 목적**
  - 원본 데이터 보존
  - 추후 실버/골드 레이어에서 자유롭게 가공할 수 있도록 재료를 보관

브론즈 레이어의 특징은 다음과 같습니다.

- 데이터 품질이 아직 완전하지 않을 수 있습니다.
  - 데이터 타입 불일치 - 숫자인데 문자열로 들어온 경우 등
  - 결측치(null) 또는 0, 공백 값 존재
  - 비즈니스 규칙이 전혀 적용되지 않은 상태
- 컬럼도 있는 그대로들어오므로,
  - 분석에 필요 없는 열/행이 섞여 있을 수 있습니다.
  - 특정 값이 "No internet service" 같이 다소 애매한 문자열로 표현되기도 합니다.

> 이 Notebook에서는 **AIDP에 등록된 브론즈 카탈로그/볼륨에 있는 Telco Churn 이력 데이터를 읽어온 뒤, 최소한의 정제를 거쳐 실버(Silver) 레이어로 옮기는 과정**을 실습합니다.

# 2. 브론즈 → 실버로 정제해야 하는 이유

브론즈 레이어는 원본 보관이 목적이기 때문에, 바로 분석이나 머신러닝 모델 학습에 쓰기에는 여러 가지 불편한 점들이 있습니다.  
그래서 브론즈 → 실버 단계에서 아래와 같은 정제가 필요합니다.

| 정제 포인트             | 문제 예시                                   | 실버 단계 처리 방향                              |
| ---------------------- | ------------------------------------------ | ---------------------------------------------- |
| **불필요한 문구 포함**   | "No internet service"처럼 의미상 “없음”이지만 긴 문자열 | "No" 등 단일 표현으로 통일 |
| **결측치 처리 필요**     | totalcharges 가 null 또는 0 으로 들어온 고객      | 평균/중앙값 치환 또는 행 제거 |
| **비즈니스 룰 미적용**    | 너무 오래된 고객 이력 포함                      | 최근 N년만 남기고 필터링 |
| **불필요/중복 컬럼 포함** | 시스템 내부용 컬럼 포함                          | 모델에 필요한 주요 컬럼만 선택 |
| **범주형 문자열 존재**    | 계약/요금/서비스 등 문자열을 모델에서 그대로 사용 불가 | LabelEncoding / OneHotEncoding 등 숫자 인코딩 적용 |

**실버(Silver) 레이어**는 이렇게 정제를 완료한 데이터를 저장하는 단계입니다.
- 브론즈에서 가져온 데이터를  
  - 타입을 맞추고  
  - 불필요한 문자열을 치환하고  
  - 필요한 컬럼과 기간만 남기고  
  - 모델링/분석에 바로 사용할 수 있는 형태로 정리합니다.
- 이후 골드(Gold) 레이어에서는  
  - 이 실버 데이터를 기반으로  
  - 실제 예측 결과를 만들어  
  - 비즈니스에 활용할 수 있게 됩니다.

# 3. 해당 Notebook : 브론즈 → 실버, Telco churn 이력 데이터 적재

`사용 파일 : TelcoCustomerChurnHistorical.csv 기본 고객 데이터`

이 Notebook에서 수행하는 전체 흐름은 다음과 같습니다.

1. **브론즈 Volume에서 TelcoCustomerChurnHistorical.csv 읽기**
  - AIDP에 등록된 브론즈 Volume 경로에서 CSV 파일을 Spark DataFrame으로 읽어옵니다.
  - 스키마를 명시적으로 지정해 이후 타입 문제를 방지합니다.

2. **브론즈 데이터 이해하기**
  - 고객 기본 정보, 계약 및 요금 정보, 서비스 이용 정보, 이탈 여부 등 각 컬럼의 의미를 확인합니다.

3. **필요한 컬럼만 선택하고 최근 연도 데이터만 필터링**
  - 분석과 모델링에 필요한 핵심 컬럼만 남깁니다.
  - 오래된 이력은 제외하여 최근 패턴 중심의 데이터셋을 만듭니다.

4. **Silver Volume 또는 카탈로그에 Parquet 또는 Delta 포맷으로 저장**
  - target_type이 file 또는 table인지에 따라 Volume 경로에 파일로 저장하거나 Data Catalog에 테이블로 등록합니다.
  - target_format이 parquet 또는 delta인지에 따라 저장 포맷이 결정됩니다.

5. **저장된 데이터 다시 읽어 검증**
  - 저장한 실버 데이터를 다시 Spark로 읽어 컬럼 구성, 데이터 건수, 샘플 데이터를 확인합니다.


> 이 노트북은 브론즈에서 실버로 데이터를 인제스트하고 기본 정제를 수행하는 단계입니다.  
> 이후 실버 데이터는 다음 단계에서 머신러닝 모델 학습에 활용됩니다.

In [1]:
# 파라미터 설정 (워크플로(Job)에서 파라미터로 받을 수 있도록 설정)
target_type   =oidlUtils.parameters.getParameter("TARGET_TYPE", "table")
target_format =oidlUtils.parameters.getParameter("TARGET_FORMAT", "parquet")

# 브론즈/실버 volume 경로 설정
bronze_folder_path = "/Volumes/demo_telco_churn_bronze/telco_churn_historical/bronze_telco_churn_historical"
silver_folder_path = "/Volumes/demo_telco_churn_silver/telco_churn_historical/silver_telco_churn_historical"

#카탈로드/스키마 이름 설정
bronze_catalog    = "demo_telco_churn_bronze"
silver_catalog    = "demo_telco_churn_silver"

bronze_schema     = "telco_churn_historical"
silver_schema     = "telco_churn_historical"

# Step 1 - Spark DataFrame으로 CSV 읽기

이 단계에서는 브론즈 레이어(Volume)에 저장된 `TelcoCustomerChurnHistorical.csv` 파일을 Spark DataFrame으로 읽어옵니다.

- AIDP 환경에서는 **브론즈 Volume 경로**가 이미 등록되어 있어서,
  - 로컬 파일처럼 직접 경로를 적는 것이 아니라
  - `/Volumes/...` 형태의 경로를 사용합니다.
- 스키마를 **명시적으로 정의**하고 나서 읽어오면,
  - `inferSchema`로 자동 추론할 때 발생할 수 있는 타입 불일치를 줄일 수 있습니다.
  - 예:&nbsp;`totalcharges` 가 일부 문자열/숫자 혼합으로 들어오는 문제 등 방지

> 이 셀을 실행하면 `telco_custchurn_history_df` 라는 브론즈 DataFrame이 생성되고,  
> 이후 단계에서 이 DataFrame을 기준으로 컬럼 선택, 필터링, 저장을 진행하게 됩니다.

In [1]:
# PySpark 타입 import
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

# Telco Churn 스키마 정의
# CSV 읽을 때 컬럼 이름과 타입을 명시적으로 고정 : 나중에 스키마 추론(inferSchema)에서 타입 꼬이는 걸 방지
telco_custchurn_history_schema = StructType(fields=[StructField("customerid", StringType(), False),
                                     StructField("gender", StringType(), True),
                                     StructField("seniorcitizen", IntegerType(), True),
                                     StructField("partner", StringType(), True),
                                     StructField("dependents", StringType(), True),
                                     StructField("tenure", IntegerType(), True),
                                     StructField("phoneservice", StringType(), True),
                                     StructField("multiplelines", StringType(), True),
                                     StructField("internetservice", StringType(), True),
                                     StructField("onlinesecurity", StringType(), True),
                                     StructField("onlinebackup", StringType(), True),
                                     StructField("deviceprotection", StringType(), True),
                                     StructField("techsupport", StringType(), True),
                                     StructField("streamingtv", StringType(), True),
                                     StructField("streamingmovies", StringType(), True),
                                     StructField("contract", StringType(), True),
                                     StructField("paperlessbilling", StringType(), True),
                                     StructField("paymentmethod", StringType(), True),
                                     StructField("monthlycharges", DoubleType(), True),
                                     StructField("totalcharges", DoubleType(), True),
                                     StructField("churn", StringType(), True),
                                     StructField("year", IntegerType(), True),
                                     StructField("exchangeid", StringType(), True),
                                     StructField("switchreg", StringType(), True)
])

# 브론즈 Volume에서 CSV 읽기
telco_custchurn_history_df = spark.read \
.option("header", True) \
.schema(telco_custchurn_history_schema) \
.csv(f"{bronze_folder_path}/TelcoCustomerChurnHistorical.csv")

# Step 2 - telco_custchurn_history_df 데이터

이 데이터는 통신사의 고객 이탈 분석을 위한 고객 이력 데이터 셋입니다.  
각 고객이 어떤 서비스를 얼마나 사용했고, 얼마나 요금을 냈으며, 결국 이탈했는지를 기록한 고객 단위 이력 테이블입니다.

- **각 행 row** : 한 명의 고객
- **각 컬럼 column** : 고객의 속성, 서비스 이용 패턴, 요금, 이탈 여부 등

편하게 이해하기 위해 컬럼들을 네 그룹으로 나눠보겠습니다.

### 1. 고객 기본 정보

고객의 인구통계학적 속성으로, 고객 세그먼트 분석의 기본이 되는 정보입니다.

| 컬럼              | 예시           | 의미               |
| ----------------- | -------------- | ------------------ |
| `customerid`      | `1423-BMPBQ`   | 고객 고유 ID        |
| `gender`          | `Female`       | 고객 성별           |
| `seniorcitizen`   | `0`            | 시니어 고객 여부    |
| `partner`         | `Yes`          | 배우자 유무         |
| `dependents`      | `No`           | 부양가족 유무       |

### 2. 계약 및 요금 정보

고객이 어떤 방식으로 계약했고 얼마나 유지했는지, 어떤 요금 구조를 사용하는지를 나타냅니다.

- `tenuremonth` : 가입 후 유지 개월 수
- `contract` : 계약 형태
- `paperlessbilling` : 종이 청구서 여부
- `paymentmethod` : 결제 수단
- `monthlycharges` : 월 청구 금액
- `totalcharges` : 누적 청구 금액

### 3. 서비스 이용 정보

인터넷, 전화, TV, 보안 서비스 등 실제로 어떤 서비스를 쓰는지에 대한 정보입니다.

- `phoneservice`, `multiplelines`
- `internetservice`
- `onlinesecurity`, `onlinebackup`, `deviceprotection`, `techsupport`
- `streamingtv`, `streamingmovies`

일부 값은 `"No internet service"`처럼  
실제로는 서비스를 사용하지 않는다는 뜻인데 문자열이 길게 표현되어 있습니다.  
실버 단계에서 이런 값들은 `"No"` 등으로 정리하게 됩니다.

### 4. 이탈 여부

- `churnvalue` : 이탈 여부를 0 또는 1 로 표현한 컬럼
- `churnlabel` : Yes 또는 No 형태의 라벨
- `churnscore`, `churnreason` 등 : 이탈 점수 및 이탈 사유

> 이 Step에서는 브론즈 데이터에 어떤 정보가 들어있는지 파악하는 것이 목표입니다.  
> 다음 Step에서 이 중 어떤 컬럼을 실버 레이어에 남길지 선택하게 됩니다.


In [1]:
telco_custchurn_history_df.show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+----+----------+---------+
|customerid|gender|seniorcitizen|partner|dependents|tenure|phoneservice|   multiplelines|internetservice|     onlinesecurity|       onlinebackup|   deviceprotection|        techsupport|        streamingtv|    streamingmovies|      contract|paperlessbilling|       paymentmethod|monthlycharges|totalcharges|churn|year|exchangeid|switchreg|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+----+-----

# Step 3 - 필요한 컬럼 + 최근 연도 데이터만 남기기

이 단계는 브론즈 데이터-`telco_custchurn_history_df`에서 **분석/모델링에 꼭 필요한 컬럼만 남기고 불필요한 컬럼은 제거**하는 과정입니다.

- 브론즈에는 원천 시스템에서 온 컬럼이 모두 들어있기 때문에
  - 분석에 직접 쓰이지 않는 내부용 필드나,
  - 중복 정보나,
  - 위치 식별자 등은 과감히 제거합니다.
- 이 Notebook에서는
  - 약 22개 컬럼을 남기고,
  - 지역 식별자 등 2개 정도 컬럼을 제거하는 식으로 스키마를 정리합니다.

또한, 브론즈 테이블에는 여러 연도의 이력이 섞여 있을 수 있기 때문에 실버 레이어에서는 **최근 데이터 위주로 분석할 수 있도록 연도 필터**를 적용합니다.

- 예:`year >= 2020` 인 데이터만 남기기
- 최근 1~2년 간의 고객 이탈 패턴에 집중하고 싶은 경우 유용

> 이 단계까지 수행하면,  
> “Telco 고객 이력 전체”였던 브론즈 데이터가  
> “모델 학습에 바로 투입 가능한 크기와 컬럼 구성을 가진 실버 후보 데이터”로 정리됩니다.


In [1]:
# 필요한 컬럼만 Select
from pyspark.sql.functions import col 
telco_custchurn_history_df=telco_custchurn_history_df.select(
    col("customerid"), 
    col("gender"), 
    col("seniorcitizen"), 
    col("partner"), 
    col("dependents"), 
    col("tenure"), 
    col("phoneservice"), 
    col("multiplelines"), 
    col("internetservice"), 
    col("onlinesecurity"), 
    col("onlinebackup"), 
    col("deviceprotection"), 
    col("techsupport"), 
    col("streamingtv"),
    col("streamingmovies"), 
    col("contract"),
    col("paperlessbilling"), 
    col("paymentmethod"), 
    col("monthlycharges"), 
    col("totalcharges"), 
    col("churn"), 
    col("year")
)

# 최근 연도 필터링
telco_custchurn_history_df=telco_custchurn_history_df.filter(
    telco_custchurn_history_df["year"]>2022
)

telco_custchurn_history_df.count()

9958

Notebook 실행 시 저장 방식을 유연하게 바꿀 수 있도록 **AIDP의 파라미터 기능**을 사용합니다.

여기서 `oidlUtils.parameters.getParameter()` 는 AIDP 워크플로(Job)가 실행될 때 **외부에서 값을 주입받을 수 있도록 해주는 함수**입니다.

즉, 같은 Notebook이라도 다음과 같이 다르게 동작시킬 수 있습니다.

- 개발 환경 : `target_type = "file"`, `target_format = "parquet"`
- 운영 환경 : `target_type = "table"`, `target_format = "delta"`

파라미터를 지정하지 않으면 기본값으로

- `target_type = "table"`
- `target_format = "parquet"`

이 적용되도록 설정해 두었습니다.  
따라서 Notebook을 직접 실행하는 경우에는 **카탈로그(Catalog)에 Parquet 테이블로 저장하는 모드**로 동작하게 됩니다.

정리하면,

- `target_type` → “어디에 저장할까?”
  - `file` : Volume에 파일로 저장
  - `table` : Data&nbsp;Catalog에 등록된 테이블로 저장
- `target_format` → “어떤 형식으로 저장할까?”
  - `parquet` : 일반적인 컬럼 기반 파일 포맷
  - `delta` : 버전 관리와 트랜잭션을 지원하는 Delta 테이블 포맷

이 두 값은 **데이터 한 줄 한 줄마다 달라지는 속성이 아니라, 전체 DataFrame을 저장할 때 한 번만 적용되는 전역 설정값**입니다.

> 비유하자면 `telco_custchurn_history_df`는 “완성된 문서”이고,  
> `target_type`/`target_format`은  
> “이 문서를 어디에, 어떤 형식으로 저장할까?” 를 정하는 **저장 옵션**입니다.  
> Word 문서를 C&nbsp;드라이브에 `.docx` 로 저장할 수도 있고,  
> Google&nbsp;Drive에 `.pdf` 로 저장할 수도 있지만  
> **문서 내용은 동일**한 것과 같은 개념입니다.


# Step 4 - 브론즈 → 실버 저장 및 검증 단계

지금까지 정제한 `telco_custchurn_history_df` DataFrame을 **실버 레이어에 실제로 저장(write)** 하고,  
그 결과를 다시 읽어서(read) **정상적으로 저장되었는지 검증(display)** 하는 단계입니다.

## (1) Silver 레이어 데이터 저장

### file 모드

- `target_type = "file"`, `target_format = "parquet"` 인 경우
  - Silver Volume 경로-`silver_folder_path`에 Parquet 파일로 저장합니다.
  - 경로 예시  
    `/Volumes/demo_telco_churn_silver/telco_churn_historical/silver_telco_churn_historical/telco_custchurn_history/`
  - 주로
    - 빠르게 실험해보고 싶을 때,
    - 파일 단위로 간단히 결과를 확인할 때 유용합니다.

### table 모드

- `target_type = "table"` 인 경우
  - Data&nbsp;Catalog에 테이블로 등록하여 저장합니다.
  - `target_format` 에 따라
    - `parquet` 테이블 또는
    - `delta` 테이블로 저장됩니다.
  - 예시 테이블 이름  
    `demo_telco_churn_silver.telco_churn_historical.telco_custchurn_history_par`

이렇게 저장해두면, 단순 파일이 아니라 **스키마와 메타데이터를 가진 테이블**로서 관리할 수 있게 됩니다.

---

#### Volume vs. Catalog

똑같은 데이터를 왜 두 군데-볼륨/카탈로그로 나누어 저장할까요?  
간단히 말하면 **볼륨은 물리적 저장소-폴더, 카탈로그는 논리적 저장소-데이터베이스**입니다.

| 개념          | 예시                                                                | 접근 방식                                       |
| ------------- | ------------------------------------------------------------------- | ---------------------------------------------- |
| **Volume**    | `/Volumes/demo_telco_churn_silver/.../data.parquet`                | 파일 경로로 접근 `spark.read.parquet(...)`     |
| **Catalog**   | `demo_telco_churn_silver.telco_churn_historical.customer_table`    | SQL 처럼 접근 `SELECT * FROM ...`              |

#### 같은 데이터라도, "누가 접근하느냐"에 따라 선택이 달라집니다.

| 관점         | Volume 방식(file)                                     | Catalog 방식(table)                                             |
| ------------ | ------------------------------------------------------ | --------------------------------------------------------------- |
| 개발자       | `spark.read.parquet` 로 파일 경로를 직접 다루는 것 선호 | `spark.read.table("schema.table")` 로도 접근 가능               |
| 분석가       | ❌ 파일 경로를 알아야 하고 Spark 코드가 필요           | ✅ 그냥 SQL로 `SELECT * FROM ...` 하면 바로 조회 가능           |
| 운영 관점    | 백업/복구 시 파일 단위 관리                             | 권한 제어, 스키마 관리, 변경 이력 관리 등 메타데이터 기반 운영 가능 |

그래서 AIDP에서는

1. **실제 데이터 파일은 Volume 내부**에 저장하고,
2. **그 경로와 스키마 정보를 Catalog 메타데이터에 등록**하여,
3. 사용자는 **파일 경로를 몰라도 `schema.table` 이름만으로 데이터에 접근**할 수 있게 합니다.


#### 왜 이렇게 나누었을까?

데이터를 단순한 “파일 모음”이 아닌, 권한, 스키마, 버전 정보를 가진 “데이터 자산”으로 관리하기 위해서입니다.
| 구분    | 파일 저장(Volume) | 테이블 저장(Catalog)         |
| ----- | ------------- | ----------------------- |
| 목적    | 빠른 실험, 임시 저장  | 운영, 분석, 공유              |
| 관리 대상 | 단순 데이터 파일     | 데이터 + 스키마 + 버전/메타데이터    |
| 장점    | 구조 단순, 빠르게 저장 | SQL 접근, 권한 관리, 메타데이터 통합 |
| 단점    | 경로 직접 관리 필요   | 초기 등록 작업 필요             |


In [1]:
if target_type == 'file':
    if target_format == 'parquet':
        (
            telco_custchurn_history_df.write
            .mode("overwrite")
            .parquet(f"{silver_folder_path}/telco_custchurn_history")
        )

elif target_type == 'table':
    if target_format == 'parquet':
        (
            telco_custchurn_history_df.write
            .mode("overwrite")
            .format("parquet")
            .saveAsTable(f"{silver_catalog}.{silver_schema}.telco_custchurn_history_par")
        )
    elif target_format == 'delta':
        (
            telco_custchurn_history_df.write
            .mode("overwrite")
            .format("delta")
            .saveAsTable(f"{silver_catalog}.{silver_schema}.telco_custchurn_history_dlt")
        )

## (2) 저장된 데이터 다시 읽기

In [1]:
# Silver 레이어에서 읽기

if target_type == 'file':
    if target_format == 'parquet':
        telco_custchurn_history_read_df = spark.read.parquet(f"{silver_folder_path}/telco_custchurn_history")
elif target_type == 'table':
    if target_format == 'parquet':
        telco_custchurn_history_read_df = spark.read.table(f"{silver_catalog}.{silver_schema}.telco_custchurn_history_par")
    elif  target_format == 'delta':
        telco_custchurn_history_read_df = spark.read.table(f"{silver_catalog}.{silver_schema}.telco_custchurn_history_dlt")

## (3) 결과 확인

In [1]:
# 결과 표시
display(telco_custchurn_history_read_df)